<a href="https://colab.research.google.com/github/Kadenthebob/UCLA-AI-PROJECT/blob/main/AutoComplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_ID = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID).to('cuda')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import gradio as gr

optionList = []

def generate(input_text):
  model_inputs = tokenizer(input_text, return_tensors='pt').to('cuda')

  output = model.generate(
      **model_inputs,
      max_new_tokens=75,
      do_sample=True,
      top_p=0.9,
      top_k=85,
      temperature=0.95,
      num_return_sequences=2,
  )
  text = tokenizer.decode(output[0], skip_special_tokens=True)
  text2 = tokenizer.decode(output[1], skip_special_tokens=True)
  return [text, text2]


def options(text):
    c = generate(text)
    optionList.append(c[0])
    optionList.append(c[1])
    return "Choice 1: " + optionList[0] + "\n"*2 + "Choice 2: " + optionList[1]



def choice(choice):
    if choice == "Choice 1":
        text = optionList[0]
    elif choice == "Choice 2":
        text = optionList[1]
    optionList.clear()
    return "You chose " + choice + "\n"*2 + text

with gr.Blocks() as demo:
    fn = choice
    input_textbox = gr.Textbox(label="Sentence Starter")
    output_textbox = gr.Textbox(label="Autocomplete Output")
    generate_button = gr.Button("Generate Autocomplete")
    generate_button.click(options, inputs=input_textbox, outputs=output_textbox)
    choice1_button = gr.Button("Choice 1")
    choice2_button = gr.Button("Choice 2")
    # Pass the actual button component to the click method
    choice1_button.click(choice, choice1_button, output_textbox)
    choice2_button.click(choice, choice2_button, output_textbox)



demo.launch()